## Off the shelf results with T5

In [7]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

In [54]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-base')
base_tokenizer = T5Tokenizer.from_pretrained('t5-base')

## Abstractive Summarization

In [31]:
text_to_summarize ="""Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, 
Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics 
at Johns Hopkins University before transitioning to education. He spent several years conducting lectures 
on data science at Johns Hopkins University and at the General Assembly before founding his own startup, 
Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. 
After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on 
his fast-growing company, while creating educational material for data science.
"""

preprocess_text = text_to_summarize.strip().replace("\n","")

print ("original text preprocessed:\n", preprocess_text)

original text preprocessed:
 Sinan Ozdemir is a data scientist, startup founder, and educator living in the San Francisco Bay Area with his dog, Charlie; cat, Euclid; and bearded dragon, Fiero. He spent his academic career studying pure mathematics at Johns Hopkins University before transitioning to education. He spent several years conducting lectures on data science at Johns Hopkins University and at the General Assembly before founding his own startup, Kylie.ai, which uses artificial intelligence to build chatbots from historical transcripts. After completing a Fellowship at the Y Combinator accelerator, Sinan spent most of his time working on his fast-growing company, while creating educational material for data science.


In [34]:
# known prompt for summarization with T5
t5_prepared_text = "summarize: " + preprocess_text

input_ids = base_tokenizer.encode(t5_prepared_text, return_tensors="pt")

# summmarize 
summary_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    min_length=30,
    max_length=50,
    early_stopping=True
)

output = base_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

print (f"Summarized text: \n{output}")

Summarized text: 
Sinan Ozdemir is a data scientist, startup founder, and educator. he founded his own startup, Kylie.ai, which uses artificial intelligence to build chatbots.


## English -> German Translation

In [11]:
input_ids = base_tokenizer.encode('translate English to German: Where is the chocolate?', return_tensors='pt')

# translate 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print (f"Translated text:\n{output}")


Translated text:
Wo ist die Schokolade?


In [12]:
# pass labels in to calculate loss

input_ids = base_tokenizer('translate English to German: Where is the chocolate?', return_tensors='pt').input_ids
labels = base_tokenizer('Wo ist die Schokolade?', return_tensors='pt').input_ids

loss = base_model(input_ids=input_ids, labels=labels).loss

labels, loss

(tensor([[ 3488,   229,    67, 31267,    58,     1]]),
 tensor(0.1136, grad_fn=<NllLossBackward0>))

## CoLA: The Corpus of Linguistic Acceptability
checking for grammatical correctess

In [13]:
input_ids = base_tokenizer.encode('cola sentence: Where is the chocolate?', return_tensors='pt')

# CoLA 
translate_ids = base_model.generate(
    input_ids,
    num_beams=4,
    no_repeat_ngram_size=3,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")


is grammatically correct?: 
acceptable


In [14]:
input_ids = base_tokenizer.encode('cola sentence: Where be a chocolates?', return_tensors='pt')

# summmarize 
translate_ids = base_model.generate(
    input_ids,
    max_length=20,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"is grammatically correct?: \n{output}")

is grammatically correct?: 
unacceptable


## STSB - Semantic Text Similarity Benchmark
Are two sentences semantically similar

In [15]:
sentence_one = 'How to fish'
sentence_two = 'Fishing Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity 
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

semantically similar? (1-5): 
3.2


In [16]:
sentence_one = 'How to fish'
sentence_two = 'Hiking Manual for beginnners'


input_ids = base_tokenizer.encode(f"stsb sentence1: {sentence_one} sentence2: {sentence_two}", return_tensors='pt')

# calculate semantic similarity
translate_ids = base_model.generate(
    input_ids,
    max_length=3,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"semantically similar? (0-5): \n{output}")

semantically similar? (1-5): 
0.4


## MNLI - Multi-Genre Natural Language Inference
Whether a premise implies (“entailment”), contradicts (“contradiction”), or neither (“neutral”) a hypothesis.

In [17]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I am running for mayor', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
entailment


In [18]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I do not really vote', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
contradiction


In [19]:
input_ids = base_tokenizer.encode(
    'mnli premise: I am active in politics. hypothesis: I code for a living', return_tensors='pt'
)

# mnli 
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
neutral


## Q/A - Question/Answering

In [22]:
input_ids = base_tokenizer.encode(
    'question: Where does Sinan live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
California


In [35]:
input_ids = base_tokenizer.encode(
    'question: Where does Matt live? context: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
Boston


In [28]:
# Sanity check with random prompts

input_ids = base_tokenizer.encode(
    'prompt1: Where does Matt live? prompt2: Sinan lives in California but Matt lives in Boston.', return_tensors='pt'
)

# Q/A
translate_ids = base_model.generate(
    input_ids,
    early_stopping=True
)

output = base_tokenizer.decode(translate_ids[0], skip_special_tokens=True)

print(f"Response: \n{output}")

Response: 
prompt2


## Fine-tuning T5 for abstractive summarization

In [37]:
from transformers import pipeline, T5ForConditionalGeneration, TrainingArguments, Trainer, \
                         DataCollatorForSeq2Seq
import pandas as pd
from datasets import Dataset
import random

In [51]:
base_model = T5ForConditionalGeneration.from_pretrained('t5-small')
base_tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [38]:
# https://www.kaggle.com/snap/amazon-fine-food-reviews?select=Reviews.csv

reviews = pd.read_csv('../data/reviews.csv')

# Pre-processing step
# Punctuation is important in grammar and important for complex decoding architectures to know when to stop!
def add_punc(s):
    if s[-1] not in ('.', '!', '?'):
        s = s + '.'
    return s

reviews.dropna(inplace=True)

reviews['Summary'] = reviews['Summary'].map(add_punc)

print(reviews.shape)

reviews.head()

(96486, 3)


,Text,Summary,Score
0,Great taffy at a great price. There was a wid...,Great taffy.,5
1,This taffy is so good. It is very soft and ch...,"Wonderful, tasty taffy.",5
2,Right now I'm mostly just sprouting this so my...,Yay Barley.,5
3,This is a very healthy dog food. Good for thei...,Healthy Dog Food.,5
4,good flavor! these came securely packed... the...,fresh and greasy!,4


In [39]:
reviews = reviews[(reviews['Summary'].str.len() < 100) & (reviews['Summary'].str.len() >=30)]

reviews.shape

(13073, 3)

In [40]:
random.seed(0)

reviews_dataset = Dataset.from_pandas(reviews.astype(str).sample(5000))

In [41]:
# We have a prompt but only as a prefix in the encoder
prefix = "summarize: "

# we will manually add our own labels because unlike GPT, we cannot assume the labels are based on the inputs
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["Text"]]
    model_inputs = base_tokenizer(inputs, max_length=1024, truncation=True)

    labels = base_tokenizer(examples["Summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [42]:
tokenized_reviews_dataset = reviews_dataset.map(preprocess_function, batched=True)

  0%|          | 0/5 [00:00<?, ?ba/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [105]:
tokenized_reviews_dataset = tokenized_reviews_dataset.train_test_split(test_size=.1)

In [106]:
# Data collator specifically for generic sequence to sequence tasks
# Use when we are translating one sequence to another like translation, summarization, etc
data_collator = DataCollatorForSeq2Seq(tokenizer=base_tokenizer, model=base_model)

In [108]:
training_args = TrainingArguments(
    output_dir="./t5_summary_results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=20,
    load_best_model_at_end=True,
    logging_steps=50,
    save_strategy='epoch'
)

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=tokenized_reviews_dataset["train"],
    eval_dataset=tokenized_reviews_dataset["test"],
    data_collator=data_collator,
)

trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, Summary, Text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 3.532301187515259,
 'eval_runtime': 38.495,
 'eval_samples_per_second': 12.989,
 'eval_steps_per_second': 0.416}

In [109]:
trainer.train()  # total of 9 hours of training on my laptop!

The following columns in the training set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, Summary, Text.
/opt/homebrew/Caskroom/miniforge/base/envs/pytorch_env/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4500
  Num Epochs = 20
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 2820


Epoch,Training Loss,Validation Loss
1,3.563800,3.173784
2,3.391200,3.113433
3,3.289800,3.072913
4,3.288000,3.041173
5,3.176900,3.020081
6,3.187400,2.997320
7,3.112700,2.977490
8,3.047000,2.964586
9,3.034600,2.953549
10,3.034000,2.941815


The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, Summary, Text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to ./t5_summary_results/checkpoint-141
Configuration saved in ./t5_summary_results/checkpoint-141/config.json
Model weights saved in ./t5_summary_results/checkpoint-141/pytorch_model.bin
tokenizer config file saved in ./t5_summary_results/checkpoint-141/tokenizer_config.json
Special tokens file saved in ./t5_summary_results/checkpoint-141/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, Summary, Text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to ./t5_summary_results/checkpoint-282
Configuration saved in ./t5_summary_re

tokenizer config file saved in ./t5_summary_results/checkpoint-1833/tokenizer_config.json
Special tokens file saved in ./t5_summary_results/checkpoint-1833/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, Summary, Text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32
Saving model checkpoint to ./t5_summary_results/checkpoint-1974
Configuration saved in ./t5_summary_results/checkpoint-1974/config.json
Model weights saved in ./t5_summary_results/checkpoint-1974/pytorch_model.bin
tokenizer config file saved in ./t5_summary_results/checkpoint-1974/tokenizer_config.json
Special tokens file saved in ./t5_summary_results/checkpoint-1974/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, S

TrainOutput(global_step=2820, training_loss=3.0722037714423864, metrics={'train_runtime': 32279.8447, 'train_samples_per_second': 2.788, 'train_steps_per_second': 0.087, 'total_flos': 1246064474849280.0, 'train_loss': 3.0722037714423864, 'epoch': 20.0})

In [110]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: Score, __index_level_0__, Summary, Text.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 32


{'eval_loss': 2.899169921875,
 'eval_runtime': 37.0292,
 'eval_samples_per_second': 13.503,
 'eval_steps_per_second': 0.432,
 'epoch': 20.0}

In [111]:
trainer.save_model()

Saving model checkpoint to ./t5_summary_results
Configuration saved in ./t5_summary_results/config.json
Model weights saved in ./t5_summary_results/pytorch_model.bin
tokenizer config file saved in ./t5_summary_results/tokenizer_config.json
Special tokens file saved in ./t5_summary_results/special_tokens_map.json


In [46]:
loaded_model = T5ForConditionalGeneration.from_pretrained('./t5_summary_results')

# summarization pipeline prepends a default prefix of summarize: 
generator = pipeline(
    'summarization', model=loaded_model, tokenizer=base_tokenizer
)

In [47]:
sam = reviews.sample(1)

print(sam['Summary'])

text = sam['Text'].tolist()[0]
text

79402    amazing, my 11 year old lab, can play with my ...
Name: Summary, dtype: object


'my 11 year old lab can once again run and play ball with my freinds. she  loves them. it like a wonder treat!!'

In [48]:
# Generate a summary
generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'my 11 year old lab can play with my freinds again!!'}]

In [50]:
# Try the base t5 on the same text
base_generator = pipeline(
    'summarization', model='t5-small', tokenizer='t5-small'
)

# Summary is a bit more extractive than our fine-tuned version and style isn't quite the same as our dataset
base_generator(text, min_length=3, max_length=15, early_stopping=True, num_beams=2)

[{'summary_text': 'my lab can run and play ball with my freinds .'}]